https://huggingface.co/docs/transformers/model_doc/speech_to_text

In [1]:
import pyaudio
p = pyaudio.PyAudio()  # Create an interface to PortAudio
p.get_default_output_device_info()

{'index': 5,
 'structVersion': 2,
 'name': 'Auriculares (WH-1000XM4)',
 'hostApi': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 2,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [2]:
for i in range(p.get_device_count()):
  dev = p.get_device_info_by_index(i)
  print((i,dev['name'],dev['maxInputChannels']))

(0, 'Asignador de sonido Microsoft - Input', 2)
(1, 'Microphone (Realtek(R) Audio)', 2)
(2, 'Auriculares con micrÃ³fono (WH-1', 1)
(3, 'SteelSeries Sonar - Microphone ', 2)
(4, 'Asignador de sonido Microsoft - Output', 0)
(5, 'Auriculares (WH-1000XM4)', 0)
(6, 'SteelSeries Sonar - Chat (Steel', 0)
(7, 'ASUS VN247 (NVIDIA High Definit', 0)
(8, 'SteelSeries Sonar - Aux (SteelS', 0)
(9, 'SteelSeries Sonar - Gaming (Ste', 0)
(10, 'Altavoces (THX Spatial)', 0)
(11, 'Altavoces (Realtek(R) Audio)', 0)
(12, 'SteelSeries Sonar - Media (Stee', 0)
(13, 'SteelSeries Sonar - Microphone ', 0)
(14, 'DELL U2421HE (Sonido Intel(R) p', 0)
(15, 'Controlador primario de captura de sonido', 2)
(16, 'Microphone (Realtek(R) Audio)', 2)
(17, 'Auriculares con micrÃ³fono (WH-1000XM4)', 1)
(18, 'SteelSeries Sonar - Microphone (SteelSeries Sonar Virtual Audio Device)', 2)
(19, 'Controlador primario de sonido', 0)
(20, 'Auriculares (WH-1000XM4)', 0)
(21, 'SteelSeries Sonar - Chat (SteelSeries Sonar Virtual Audio D

In [11]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1 # Number of audio channels (1 for mono, 2 for stereo)
fs = 16000  # Record at 44100 samples per second
seconds = 6
filename = "output_two.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [7]:
from audio2numpy import open_audio
path = "output.wav"
signal, sampling_rate = open_audio(path)

In [90]:
sampling_rate

16000

In [8]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset

C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
inputs = processor(signal, sampling_rate=sampling_rate, return_tensors="pt")
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
transcription

['to get the information from the next i p others']

In [34]:
#ds = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

In [25]:
#inputs = processor(ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="pt")

In [ ]:
model_path = "whiterabbitneo/WhiteRabbitNeo-13B-v1" # over 25GB in 6 files

***

In [16]:
ds[0]["audio"]["sampling_rate"]

16000

In [15]:
ds[0]["audio"]["array"]

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ])

+ https://pypi.org/project/audio2numpy/
+ https://realpython.com/python-speech-recognition/

Recording
Finished recording


In [5]:
!dir

 El volumen de la unidad D es Kuky
 El n£mero de serie del volumen es: 3A98-E69F

 Directorio de D:\Github\infobot

23/01/2024  10:04 p. m.    <DIR>          .
21/01/2024  06:01 p. m.    <DIR>          ..
21/01/2024  06:38 p. m.    <DIR>          .ipynb_checkpoints
23/01/2024  10:02 p. m.            11.609 boty.ipynb
23/01/2024  10:04 p. m.           880.684 output.wav
21/01/2024  06:02 p. m.                62 README.md
21/01/2024  06:38 p. m.             6.330 req.txt
               4 archivos        898.685 bytes
               3 dirs  198.792.372.224 bytes libres


In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

assistant_model_id = "distil-whisper/distil-large-v2"

assistant_model = AutoModelForCausalLM.from_pretrained(
    assistant_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
assistant_model.to(device)

model_id = "openai/whisper-large-v2"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    generate_kwargs={"assistant_model": assistant_model},
    torch_dtype=torch_dtype,
    device=device,
)

C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from audio2numpy import open_audio
path = "output.wav"
signal, sampling_rate = open_audio(path)

In [3]:
prediction = pipe(signal, batch_size=8)["text"]

In [4]:
prediction

' to get the information from the next IP address'

In [2]:
import torch, json
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_path = "whiterabbitneo/WhiteRabbitNeo-13B-v1"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=False,
    load_in_8bit=True,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)


def generate_text(instruction):
    tokens = tokenizer.encode(instruction)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to(device)

    instance = {
        "input_ids": tokens,
        "top_p": 1.0,
        "temperature": 0.5,
        "generate_len": 1024,
        "top_k": 50,
    }

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length + instance["generate_len"],
            use_cache=True,
            do_sample=True,
            top_p=instance["top_p"],
            temperature=instance["temperature"],
            top_k=instance["top_k"],
            num_return_sequences=1,
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    answer = string.split("USER:")[0].strip()
    return f"{answer}"


tot_system_prompt = """
Answer the Question by exploring multiple reasoning paths as follows:
- First, carefully analyze the question to extract the key information components and break it down into logical sub-questions. This helps set up the framework for reasoning. The goal is to construct an internal search tree.
- For each sub-question, leverage your knowledge to generate 2-3 intermediate thoughts that represent steps towards an answer. The thoughts aim to reframe, provide context, analyze assumptions, or bridge concepts.
- Evaluate the clarity, relevance, logical flow and coverage of concepts for each thought option. Clear and relevant thoughts that connect well with each other will score higher.
- Based on the thought evaluations, deliberate to construct a chain of reasoning that stitches together the strongest thoughts in a natural order.
- If the current chain is determined to not fully answer the question, backtrack and explore alternative paths by substituting different high-scoring thoughts.
- Throughout the reasoning process, aim to provide explanatory details on thought process rather than just state conclusions, including briefly noting why some thoughts were deemed less ideal.
- Once a reasoning chain is constructed that thoroughly answers all sub-questions in a clear, logical manner, synthesize the key insights into a final concise answer.
- Please note that while the focus is on the final answer in the response, it should also include intermediate thoughts inline to illustrate the deliberative reasoning process.
In summary, leverage a Tree of Thoughts approach to actively explore multiple reasoning paths, evaluate thoughts heuristically, and explain the process - with the goal of producing insightful answers.
"""

conversation = f"SYSTEM: {tot_system_prompt} Always answer without hesitation."


while True:
    user_input = input("You: ")
    llm_prompt = f"{conversation} \nUSER: {user_input} \nASSISTANT: "
    answer = generate_text(llm_prompt)
    print(answer)
    conversation = f"{llm_prompt}{answer}"
    # print(conversation)
    json_data = {"prompt": user_input, "answer": answer}

    # print(json_data)
    # with open(output_file_path, "a") as output_file:
    #     output_file.write(json.dumps(json_data) + "\n")

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CUDA SETUP: Solution 2): If you do not have sudo rights, you can do the following:
CUDA SETUP: Solution 2a): Find the cuda library via: find / -name libcuda.so 2>/dev/null
CUDA SETUP: Solution 2b): Once the library is found add it to the LD_LIBRARY_PATH: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:FOUND_PATH_FROM_2a
CUDA SETUP: Solution 2c): For a permanent solution add the export from 2b i

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [3]:
!python -m bitsandbytes


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
False
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CUDA SETUP: Solution 2): If you do not have sudo rights, you can do the following:
CUDA SETUP: Solution 2a):

C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\bitsandbytes\cuda_setup\main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/Usuario/Anaconda3/envs/spe_bot/lib')}
  warn(msg)
C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\bitsandbytes\cuda_setup\main.py:147: UserWarning: C:\Users\Usuario\Anaconda3\envs\spe_bot did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\bitsandbytes\cuda_setup\main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
  warn(msg)
C:\Users\Usuario\Anaconda3\envs\spe_bot\lib\site-packages\bitsandbytes\cuda_setup\main.py:147: UserWarning: WARNING: The following directories listed in your path